# Using Agents for Amazon Bedrock

In this notebook we will create an Agent for Amazon Bedrock that interacts with data in DynamoDB.

With this agent, parents can check teacher availabilty for an upcoming parent teacher meeting and book an availble time with the teacher of their choice.

A Lambda function is used to check teacher availabilty and create bookings.

## Prerequisites
Update the botocore, boto3 and AWS cli packages (ignore any errors from the pip dependency resolver if they appear!)

In [ ]:
%pip install --upgrade -q botocore
%pip install --upgrade -q boto3
%pip install --upgrade -q awscli

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
from datetime import datetime, timedelta
import os
from botocore.exceptions import ClientError

In [ ]:
# Logging facility for Python is used to help confirm things are working as expected!
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Create the boto3 clients needed for a number of AWS services

In [ ]:
# creating boto3 clients for the services we'll use
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

Setting configuration variables for the session, as well as those needed for the Bedrock agent and a Lambda function that will be created later

In [ ]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

In [ ]:
# configuration variables
suffix = f"{region}-{account_id}"
agent_name = "booking-asst-function-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "Agent for booking teacher meetings"
agent_instruction = "You are a booking system agent, enabling parents of children attending Fir Tree lodge High School to book 30 minute appointments with their child's teacher on Parents' Evening"
agent_action_group_name = "BookingsActionGroup"
agent_action_group_description = "Actions for helping parents to book appointments with a teacher on Parents' Evening"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

## Creating a Lambda Function

A Lambda function is going to interact with our data in DynamoDB. This function will later be triggered by the agent.
1. `lambda_function.py` file contains the function logic
2. An IAM role is needed to give our Lambda function the required permissions 

## Creating a Dataset

We create two DynamoDB tables to store the data that our agent will interact with.

In [ ]:
import boto3
import random
from datetime import datetime, date, timedelta

# Initialize DynamoDB client
dynamodb = boto3.resource('dynamodb')

# Create teachers table
def create_teachers_table():
    table = dynamodb.create_table(
        TableName='Teachers',
        KeySchema=[
            {
                'AttributeName': 'teacher_id',
                'KeyType': 'HASH'  # Partition key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'teacher_id',
                'AttributeType': 'S'
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
    
    # Wait until the table exists
    table.meta.client.get_waiter('table_exists').wait(TableName='Teachers')
    return table

# Create appointments table
def create_appointments_table():
    table = dynamodb.create_table(
        TableName='ParentTeacherAppointments',
        KeySchema=[
            {
                'AttributeName': 'appointment_id',
                'KeyType': 'HASH'  # Partition key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'appointment_id',
                'AttributeType': 'S'
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
    
    # Wait until the table exists
    table.meta.client.get_waiter('table_exists').wait(TableName='ParentTeacherAppointments')
    return table

# Insert teacher data
def populate_teachers(table):
    teachers = [
        {
            'teacher_id': 'T001',
            'name': 'Mr Stokes',
            'subject': 'History',
            'room': 'H1'
        },
        {
            'teacher_id': 'T002',
            'name': 'Miss White',
            'subject': 'Sociology',
            'room': 'S3'
        },
        {
            'teacher_id': 'T003',
            'name': 'Mrs Willmott',
            'subject': 'I.T.',
            'room': 'IT2'
        }
    ]
    
    for teacher in teachers:
        table.put_item(Item=teacher)

# Generate available time slots
def generate_time_slots():
    meeting_date = "2025-09-23"
    start_time = datetime.strptime(f"{meeting_date} 15:30", "%Y-%m-%d %H:%M")
    end_time = datetime.strptime(f"{meeting_date} 21:00", "%Y-%m-%d %H:%M")
    
    time_slots = []
    current_time = start_time
    
    while current_time <= end_time:
        time_slots.append(current_time.strftime("%H:%M"))
        current_time += timedelta(minutes=30)
    
    return time_slots

# Generate sample appointment slots for each teacher
def populate_appointments(table):
    time_slots = generate_time_slots()
    teacher_ids = ['T001', 'T002', 'T003']
    appointment_counter = 1

    for teacher_id in teacher_ids:
        for time_slot in time_slots:
            # Create an available appointment slot
            appointment_id = f'APT{appointment_counter:04d}'
            
            table.put_item(
                Item={
                    'appointment_id': appointment_id,
                    'teacher_id': teacher_id,
                    'booking_date': '2025-09-23',
                    'time_slot': time_slot,
                    'booking_status': 'available',
                    'room': get_teacher_room(teacher_id),
                    'created_at': datetime.now().isoformat(),
                    'last_updated': datetime.now().isoformat()
                }
            )
            appointment_counter += 1

def get_teacher_room(teacher_id):
    rooms = {
        'T001': 'H1',  # Mr Stokes - History
        'T002': 'S3',  # Miss White - Sociology
        'T003': 'IT2'  # Mrs Willmott - I.T.
    }
    return rooms.get(teacher_id)

# Sample data for demonstration bookings
def create_sample_bookings(table):
    # List of sample student names and their parents
    sample_bookings = [
        {
            'student_name': 'James Wilson',
            'parent_name': 'Sarah Wilson',
            'teacher_id': 'T001',
            'time_slot': '16:00'
        },
        {
            'student_name': 'Emily Chen',
            'parent_name': 'David Chen',
            'teacher_id': 'T002',
            'time_slot': '17:30'
        },
        {
            'student_name': 'Mohammed Ahmed',
            'parent_name': 'Fatima Ahmed',
            'teacher_id': 'T003',
            'time_slot': '18:00'
        }
    ]

    for idx, booking in enumerate(sample_bookings, 1):
        appointment_id = f'APT{idx:04d}'
        
        # Update the appointment with booking details
        table.update_item(
            Key={
                'appointment_id': appointment_id
            },
            UpdateExpression='SET student_name = :s, parent_name = :p, booking_status = :st, last_updated = :lu',
            ExpressionAttributeValues={
                ':s': booking['student_name'],
                ':p': booking['parent_name'],
                ':st': 'booked',
                ':lu': datetime.now().isoformat()
            }
        )

def main():
    try:
        # Create and populate teachers table
        teachers_table = create_teachers_table()
        populate_teachers(teachers_table)
        print("Teachers table created and populated successfully")

        # Create and populate appointments table
        appointments_table = create_appointments_table()
        populate_appointments(appointments_table)
        print("Appointments table created and populated successfully")

        # Create some sample bookings
        create_sample_bookings(appointments_table)
        print("Sample bookings created successfully")

    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()


Create our lambda function. 
It has the ability to check which appointments are available at a given time and date, and book a meeting if no conflicts exist.

In [ ]:
%%writefile lambda_function.py
import os
import boto3
from datetime import datetime, timedelta
from botocore.exceptions import ClientError

def check_teacher_availability(teacher_id, booking_date, time_slot):
    """
    Check if the specified teacher is available at the given time slot.
    Returns "Available" if free, otherwise returns a message with conflict details.
    """
    dynamodb = boto3.resource('dynamodb')
    appointments_table = dynamodb.Table('ParentTeacherAppointments')
    
    try:
        response = appointments_table.scan(
            FilterExpression='teacher_id = :tid AND booking_date = :date AND time_slot = :ts',
            ExpressionAttributeValues={
                ':tid': teacher_id,
                ':date': booking_date,
                ':ts': time_slot
            }
        )
        
        appointments = response.get('Items', [])
        if not appointments:
            return "No available appointment slot found."
            
        appointment = appointments[0]
        if appointment['booking_status'] == 'booked':
            return "Not available. This time slot is already booked."
        else:
            return "Available"
            
    except ClientError as e:
        print(f"Error accessing DynamoDB: {str(e)}")
        raise

def book_appointment(teacher_id, booking_date, time_slot, student_name, parent_name):
    """
    Book an appointment with the specified teacher.
    First checks for availability and then creates the booking if possible.
    """
    dynamodb = boto3.resource('dynamodb')
    appointments_table = dynamodb.Table('ParentTeacherAppointments')
    teachers_table = dynamodb.Table('Teachers')
    
    try:
        # Verify teacher exists
        teacher_response = teachers_table.get_item(
            Key={'teacher_id': teacher_id}
        )
        if 'Item' not in teacher_response:
            return f"Teacher with ID {teacher_id} not found."
            
        teacher = teacher_response['Item']
        
        # Find the appointment slot
        response = appointments_table.scan(
            FilterExpression='teacher_id = :tid AND booking_date = :date AND time_slot = :ts',
            ExpressionAttributeValues={
                ':tid': teacher_id,
                ':date': booking_date,
                ':ts': time_slot
            }
        )
        
        appointments = response.get('Items', [])
        if not appointments:
            return "No available appointment slot found."
            
        appointment = appointments[0]
        if appointment['booking_status'] == 'booked':
            return f"Cannot book appointment with {teacher['name']} at {time_slot}. Time slot is already taken."
        
        # Update the appointment with booking details
        appointments_table.update_item(
            Key={'appointment_id': appointment['appointment_id']},
            UpdateExpression='SET student_name = :s, parent_name = :p, booking_status = :st, last_updated = :lu',
            ExpressionAttributeValues={
                ':s': student_name,
                ':p': parent_name,
                ':st': 'booked',
                ':lu': datetime.now().isoformat()
            }
        )
        
        return f"Appointment confirmed with {teacher['name']} on {booking_date} at {time_slot} in room {teacher['room']} for {student_name}."
        
    except ClientError as e:
        print(f"Error accessing DynamoDB: {str(e)}")
        raise

def get_available_slots(teacher_id=None):
    """
    Get all available appointment slots, optionally filtered by teacher.
    """
    dynamodb = boto3.resource('dynamodb')
    appointments_table = dynamodb.Table('ParentTeacherAppointments')
    
    try:
        if teacher_id:
            response = appointments_table.scan(
                FilterExpression='teacher_id = :tid AND booking_status = :st AND booking_date = :d',
                ExpressionAttributeValues={
                    ':tid': teacher_id,
                    ':st': 'available',
                    ':d': '2025-09-23'
                }
            )
        else:
            response = appointments_table.scan(
                FilterExpression='booking_status = :st AND booking_date = :d',
                ExpressionAttributeValues={
                    ':st': 'available',
                    ':d': '2025-09-23'
                }
            )
        
        slots = response.get('Items', [])
        return sorted(slots, key=lambda x: x['time_slot'])
        
    except ClientError as e:
        print(f"Error accessing DynamoDB: {str(e)}")
        raise

def lambda_handler(event, context):
    """
    Lambda handler that supports three functions:
      - 'check_teacher_availability': Checks if a teacher is available at a specific time.
      - 'book_appointment': Books an appointment with a teacher.
      - 'get_available_slots': Gets all available appointment slots.
    """
    actionGroup = event.get('actionGroup')
    function = event.get('function')
    parameters = event.get('parameters', [])
    responseBody = {
        "TEXT": {
            "body": "Error, no function was called"
        }
    }
    
    if function == 'check_teacher_availability':
        teacher_id = None
        booking_date = None
        time_slot = None
        
        for param in parameters:
            if param["name"] == "teacher_id":
                teacher_id = param["value"]
            elif param["name"] == "booking_date":
                booking_date = param["value"]
            elif param["name"] == "time_slot":
                time_slot = param["value"]
        
        # Validate parameters
        if not all([teacher_id, booking_date, time_slot]):
            raise Exception("Missing mandatory parameters")
        
        availability = check_teacher_availability(teacher_id, booking_date, time_slot)
        responseBody = {
            'TEXT': {
                "body": f"Availability for teacher {teacher_id} on {booking_date} at {time_slot}: {availability}"
            }
        }
    
    elif function == 'book_appointment':
        teacher_id = None
        booking_date = None
        time_slot = None
        student_name = None
        parent_name = None
        
        for param in parameters:
            if param["name"] == "teacher_id":
                teacher_id = param["value"]
            elif param["name"] == "booking_date":
                booking_date = param["value"]
            elif param["name"] == "time_slot":
                time_slot = param["value"]
            elif param["name"] == "student_name":
                student_name = param["value"]
            elif param["name"] == "parent_name":
                parent_name = param["value"]
        
        # Validate parameters
        if not all([teacher_id, booking_date, time_slot, student_name, parent_name]):
            raise Exception("Missing mandatory parameters")
        
        booking_result = book_appointment(teacher_id, booking_date, time_slot, student_name, parent_name)
        responseBody = {
            'TEXT': {
                "body": booking_result
            }
        }
    
    elif function == 'get_available_slots':
        teacher_id = None
        
        for param in parameters:
            if param["name"] == "teacher_id":
                teacher_id = param["value"]
        
        available_slots = get_available_slots(teacher_id)
        formatted_slots = []
        for slot in available_slots:
            formatted_slots.append(f"Teacher: {slot['teacher_id']}, Time: {slot['time_slot']}, Room: {slot['room']}")
        
        responseBody = {
            'TEXT': {
                "body": "Available slots:\n" + "\n".join(formatted_slots)
            }
        }
    
    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }
    
    function_response = {
        'response': action_response,
        'messageVersion': event.get('messageVersion')
    }
    
    print("Response:", function_response)
    return function_response


Next let's create the Lambda IAM role and policy to invoke a Bedrock model

In [ ]:
import json
import time
import boto3

iam_client = boto3.client('iam')
lambda_function_role = 'ParentTeacherBookingLambdaRole'

# Create IAM Role for the Lambda function
try:
    # Trust policy to allow Lambda to assume this role
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    # Convert the policy to JSON string
    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    # Create the IAM role
    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json,
        Description='IAM role for Parent Teacher Booking Lambda function'
    )

    # Pause to make sure role is created
    time.sleep(10)
except Exception as e:
    print(f"Role might already exist, getting existing role: {str(e)}")
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

# Attach basic Lambda execution policy for CloudWatch Logs
iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

# Create custom policy for DynamoDB access
dynamodb_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "dynamodb:GetItem",
                "dynamodb:Scan",
                "dynamodb:Query",
                "dynamodb:PutItem",
                "dynamodb:UpdateItem",
                "dynamodb:DeleteItem"
            ],
            "Resource": [
                f"arn:aws:dynamodb:*:{lambda_iam_role['Role']['Arn'].split(':')[4]}:table/Teachers",
                f"arn:aws:dynamodb:*:{lambda_iam_role['Role']['Arn'].split(':')[4]}:table/ParentTeacherAppointments"
            ]
        }
    ]
}

try:
    # Create the custom policy
    custom_policy = iam_client.create_policy(
        PolicyName='ParentTeacherBookingDynamoDBAccess',
        PolicyDocument=json.dumps(dynamodb_policy_document),
        Description='Policy for Parent Teacher Booking Lambda to access DynamoDB tables'
    )
    
    # Attach the custom policy to the role
    iam_client.attach_role_policy(
        RoleName=lambda_function_role,
        PolicyArn=custom_policy['Policy']['Arn']
    )
except iam_client.exceptions.EntityAlreadyExistsException:
    # If policy already exists, get its ARN and attach it
    account_id = lambda_iam_role['Role']['Arn'].split(':')[4]
    policy_arn = f"arn:aws:iam::{account_id}:policy/ParentTeacherBookingDynamoDBAccess"
    iam_client.attach_role_policy(
        RoleName=lambda_function_role,
        PolicyArn=policy_arn
    )
lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)
print(f"Role ARN: {lambda_iam_role['Role']['Arn']}")


We can now package the Lambda function to a Zip file and create the Lambda function using boto3

In [ ]:
# Package up the Lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

## Create Agent
We will now create the agent. To do so, we first need to create the agent policies that allow bedrock model invocation for a specific foundation model and the agent IAM role with the policy associated to it. 

In [ ]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

### Creating the agent
Once the needed IAM role is created, we use the Bedrock Agent client `create_agent` function to create the agent. 
It requires an agent name, underlying foundation model and instructions. You can also provide an agent description. 

In [ ]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

Store the agent id in a local variable to use later

In [ ]:
agent_id = response['agent']['agentId']
agent_id

## Create Agent Action Group
We will now create an agent action group that uses the Lambda function. The [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet created an agent version or alias. To inform the agent about the action group capabilities, we provide an action group description.

The Action Group functionality is defined using a `functionSchema` and we need to provide the `name`, `description` and `parameters` for each agent function.

In [ ]:
agent_functions = [
    {
        'name': 'check_teacher_availability',
        'description': 'Check if a specific teacher is available for a parent-teacher meeting time slot',
        'parameters': {
            "teacher_id": {
                "description": "The ID of the teacher (e.g., 'T001' for Mr Stokes, 'T002' for Miss White, 'T003' for Mrs Willmott)",
                "required": True,
                "type": "string"
            },
            "booking_date": {
                "description": "The date of the parent-teacher meeting (2025-09-23)",
                "required": True,
                "type": "string"
            },
            "time_slot": {
                "description": "The desired appointment time in HH:MM format (between 15:30 and 21:00)",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'book_appointment',
        'description': 'Book a 30-minute appointment with a teacher for the parent-teacher meeting',
        'parameters': {
            "teacher_id": {
                "description": "The ID of the teacher (e.g., 'T001' for Mr Stokes, 'T002' for Miss White, 'T003' for Mrs Willmott)",
                "required": True,
                "type": "string"
            },
            "booking_date": {
                "description": "The date of the parent-teacher meeting (2025-09-23)",
                "required": True,
                "type": "string"
            },
            "time_slot": {
                "description": "The desired appointment time in HH:MM format (between 15:30 and 21:00)",
                "required": True,
                "type": "string"
            },
            "student_name": {
                "description": "The full name of the student",
                "required": True,
                "type": "string"
            },
            "parent_name": {
                "description": "The full name of the parent attending the meeting",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'get_available_slots',
        'description': 'Get all available appointment slots, optionally filtered by teacher',
        'parameters': {
            "teacher_id": {
                "description": "Optional: The ID of the teacher to filter slots (e.g., 'T001' for Mr Stokes, 'T002' for Miss White, 'T003' for Mrs Willmott)",
                "required": False,
                "type": "string"
            }
        }
    },
    {
        'name': 'get_teacher_info',
        'description': 'Get information about a specific teacher including their subject and room',
        'parameters': {
            "teacher_id": {
                "description": "The ID of the teacher (e.g., 'T001' for Mr Stokes, 'T002' for Miss White, 'T003' for Mrs Willmott)",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'get_booking_details',
        'description': 'Get details of an existing booking',
        'parameters': {
            "appointment_id": {
                "description": "The unique ID of the appointment booking",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'list_teacher_schedule',
        'description': 'Get the complete schedule for a specific teacher showing all booked and available slots',
        'parameters': {
            "teacher_id": {
                "description": "The ID of the teacher (e.g., 'T001' for Mr Stokes, 'T002' for Miss White, 'T003' for Mrs Willmott)",
                "required": True,
                "type": "string"
            },
            "booking_date": {
                "description": "The date of the parent-teacher meeting (2025-09-23)",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'cancel_booking',
        'description': 'Cancel an existing appointment booking',
        'parameters': {
            "appointment_id": {
                "description": "The unique ID of the appointment booking to cancel",
                "required": True,
                "type": "string"
            },
            "parent_name": {
                "description": "The name of the parent who made the booking (for verification)",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'get_next_available_slot',
        'description': 'Find the next available appointment slot for a specific teacher',
        'parameters': {
            "teacher_id": {
                "description": "The ID of the teacher (e.g., 'T001' for Mr Stokes, 'T002' for Miss White, 'T003' for Mrs Willmott)",
                "required": True,
                "type": "string"
            },
            "after_time": {
                "description": "Find the next available slot after this time (HH:MM format)",
                "required": True,
                "type": "string"
            }
        }
    }
]

In [ ]:
# Wait for the agent to finish being created
time.sleep(30)
# Configure and create the action group
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [ ]:
agent_action_group_response

## Enable Agent to invoke the Action Group Lambda function
Enable the agent to invoke the Lambda function by adding the required permission to a resource-based policy. 

In [ ]:
# Allow invoke permission on Lambda
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)


In [ ]:
response

## Preparing the Agent

Creating a DRAFT version of the agent that can be used for internal testing.


In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

In [ ]:
# Pause to make sure agent is prepared
time.sleep(30)

# Extract the agentAliasId from the response
agent_alias_id = "TSTALIASID"

## Invoke Agent

Invoking the agent using the `bedrock-agent-runtime` client

In [ ]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="When is Miss White available",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

In [ ]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

In [ ]:
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="Book the first available appointment with Miss White, my name is Faye Ellis",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

In [ ]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

## Cleaning Up

Run the following cells to delete everything we created, including:

1. Action group
2. Agent 
3. Lambda function
4. IAM roles and policies
5. DynamoDB tables


In [ ]:
# Deleting the Action Group
action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    functionSchema={
        'functions': agent_functions
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [ ]:
# Deleting the Agent
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [ ]:
# Deleting the Lambda function
lambda_client.delete_function(
    FunctionName=lambda_function_name
)

In [ ]:
# Detaching the IAM Policies
for policy in [agent_bedrock_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
iam_client.detach_role_policy(RoleName=lambda_function_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

In [ ]:
# Deleting the IAM Roles

for role_name in [agent_role_name, lambda_function_role]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)


In [ ]:
# Deleting the DynamoDB tables

# Deleting the DynamoDB tables
for table_name in ['Teachers', 'ParentTeacherAppointments']:
    table = dynamodb.Table(table_name)
    table.delete()


## Remember to also delete the Jupyter Notebook if you no longer need it!